In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from api_keys import g_key

In [2]:
# Read city data csv into dataframe and print
city_data = pd.read_csv("city_data.csv")
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Alice Springs,0,AU,1587234135,55,-23.70,133.88,60.80,4.70
1,Rikitea,1,PF,1587234135,70,-23.12,-134.97,76.78,4.68
2,Haines Junction,1,CA,1587234136,50,60.75,-137.51,42.53,1.50
3,Ribeira Grande,75,PT,1587234136,88,38.52,-28.70,64.40,9.17
4,Hobart,40,AU,1587234136,87,-42.88,147.33,45.00,1.12


# Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Plot city location heat map with humidity as insensity
fig = gmaps.figure()
locations = city_data[["Latitude", "Longitude"]].astype(float)
weights = city_data["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=weights.max(),
                                 point_radius = 1)

# Show maps with heat layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create new DataFrame fitting weather criteria
ideal_weather = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
ideal_weather.dropna()
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
29,Rocha,0,UY,1587233889,59,-34.48,-54.33,72.12,7.54
55,Jalu,0,LY,1587233864,23,29.03,21.55,73.94,8.63
68,Jasaan,0,PH,1587234141,67,8.65,124.78,75.96,4.32
130,Angoche,0,MZ,1587234146,82,-16.23,39.91,78.80,8.16
456,Tacuarembó,0,UY,1587234174,36,-31.73,-55.98,76.75,8.79


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create new dataframe to hold hotel information
hotel_df = ideal_weather[["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""

# Set parameters to find hotels within the 5000m radius
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Make search request using the set parameters
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params).json()

     # Handle exceptions for missing results
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# Print hotel dataframe
hotel_df

C:\Users\omoto\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\omoto\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Latitude,Longitude,Hotel Name
29,Rocha,UY,-34.48,-54.33,Beleza Pura B&B
55,Jalu,LY,29.03,21.55,Jalu Hotel
68,Jasaan,PH,8.65,124.78,Kahulugan Spring Resort
130,Angoche,MZ,-16.23,39.91,Guest House HESADA
456,Tacuarembó,UY,-31.73,-55.98,Hotel Orange


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [7]:
# Create marker layer to show hotel information
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add marker and heat layers
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))